論文  
https://arxiv.org/abs/2209.14916<br>
<br>
GitHub<br>
https://github.com/GuyTevet/motion-diffusion-model<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/motions_diffusion_model_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

## 実行方法
「ランタイム」→「すべてのセルを実行」を選択

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/GuyTevet/motion-diffusion-model.git

# Commits on Oct 10, 2022使用
%cd /content/motion-diffusion-model
!git checkout c5c0ae13974c110cb6ea8292dc88da28d3d78854

## ライブラリのインストール

In [ ]:
%cd /content/motion-diffusion-model

!pip install --upgrade gdown
!pip install smplx chumpy trimesh
!pip install moviepy imageio==2.4.1
!pip install --upgrade gdown

!python -m spacy==3.4.1 download en_core_web_sm
!pip install git+https://github.com/openai/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1

## SMPL、Human3Dなどデータセットダウンロード

In [ ]:
%cd /content/motion-diffusion-model

# Download SMPL
!bash prepare/download_smpl_files.sh
!bash prepare/download_glove.sh

In [ ]:
%cd /content

# Download Human3D
!git clone https://github.com/EricGuo5513/HumanML3D.git
!unzip ./HumanML3D/HumanML3D/texts.zip -d ./HumanML3D/HumanML3D/ > /dev/null
!cp -r HumanML3D/HumanML3D motion-diffusion-model/dataset/HumanML3D

## ライブラリのインポート

In [ ]:
import os

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/motion-diffusion-model

if not os.path.exists('humanml_trans_enc_512.zip'):
  !gdown 'https://drive.google.com/uc?id=1PE0PK8e5a5j-7-Xhs5YET5U5pGh0c821'

!unzip humanml_trans_enc_512.zip -d ./pretrained/

# Text to Human Motion

In [ ]:
prompt = "The person suddenly dances while walking." #@param {type:"string"}

In [ ]:
!python -m sample \
  --model_path ./pretrained/humanml_trans_enc_512/model000200000.pt \
  --text_prompt '{prompt}'

## 出力結果を表示

In [ ]:
foldername = prompt.replace(" ", "_")
foldername = foldername.replace(".", "")

In [ ]:
base_path = f'/content/motion-diffusion-model/pretrained/humanml_trans_enc_512/samples_humanml_trans_enc_512_000200000_seed10_{foldername}/'
out_motion_video0 = base_path + "sample00_rep00.mp4"
out_motion_video1 = base_path + "sample00_rep01.mp4"
out_motion_video2 = base_path + "sample00_rep02.mp4"

In [ ]:
clip = VideoFileClip(out_motion_video0)
clip = resize(clip, height=420)
clip.ipython_display()

In [ ]:
clip = VideoFileClip(out_motion_video1)
clip = resize(clip, height=420)
clip.ipython_display()

In [ ]:
clip = VideoFileClip(out_motion_video2)
clip = resize(clip, height=420)
clip.ipython_display()

# Rendering SMPL Mesh

In [ ]:
!python -m visualize.render_mesh \
  --input_path=$out_motion_video0

## Rendering Objファイル

In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)

# add path for demo utils functions 
import sys
import os
sys.path.append(os.path.abspath(''))

!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

In [ ]:
# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

obj_filename = f'/content/motion-diffusion-model/pretrained/humanml_trans_enc_512/samples_humanml_trans_enc_512_000200000_seed10_{foldername}/sample00_rep00_obj/frame102.obj'

# Load obj file
mesh = load_objs_as_meshes([obj_filename], device=device)

In [ ]:
verts, faces_idx, _ = load_obj(obj_filename)
faces = faces_idx.verts_idx

# Initialize each vertex to be white in color.
verts_rgb = torch.ones_like(verts)[None]  # (1, V, 3)
textures = TexturesVertex(verts_features=verts_rgb.to(device))

# Create a Meshes object
mesh = Meshes(
    verts=[verts.to(device)],   
    faces=[faces.to(device)],
    textures=textures
)

# Render the plotly figure
fig = plot_scene({
    "Human motion": {
        "person": mesh
    }
})
fig.show()